# Dependencies  

In [ ]:
from herbie import Herbie
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import traceback
import s3fs
import numcodecs as ncd
import numpy as np
import geopandas as gpd
import pandas as pd
import datetime
from datetime import timedelta
import xarray as xr
import math
from shapely.ops import unary_union
from shapely.geometry import Point, LineString
import time
from datetime import datetime
from dateutil.relativedelta import relativedelta

# resources:
# https://mesowest.utah.edu/html/hrrr/zarr_documentation/html/ex_python_plot_zarr.html
# https://home.chpc.utah.edu/~u0553130/Brian_Blaylock/HRRR_archive/hrrr_sfc_table_f02-f18.html
# https://www.nco.ncep.noaa.gov/pmb/products/hrrr/ 


# Weather data for lat long date hour

In [ ]:
#location:  Can only do coordinates in area that is covered by the HRRR CONUS grid.
point_lat = 40.7608
point_lon = -113.8910

# Date: [YYYYMMDD] must be in this format. Example is for Jan 8, 2021
date = '20200917'

# Hour: [00-23]z must be in the two digit format  i.e. 06 or 19
hr = '00'

# Level: 
level = 'surface'

# Variable:
var = 'GUST'

data_url = f'hrrrzarr/sfc/{date}/{date}_{hr}z_fcst.zarr/{level}/{var}/{level}/{var}/'

fs = s3fs.S3FileSystem(anon=True)

chunk_index = xr.open_zarr(s3fs.S3Map("s3://hrrrzarr/grid/HRRR_chunk_index.zarr", s3=fs))
###
###
###
# This is the projection the HRRR grid uses.
projection = ccrs.LambertConformal(central_longitude=262.5, 
                                   central_latitude=38.5, 
                                   standard_parallels=(38.5, 38.5),
                                    globe=ccrs.Globe(semimajor_axis=6371229,
                                                     semiminor_axis=6371229))

x, y = projection.transform_point(point_lon, point_lat, ccrs.PlateCarree())

nearest_point = chunk_index.sel(x=x, y=y, method="nearest")
fcst_chunk_id = f"0.{nearest_point.chunk_id.values}"
#print(fcst_chunk_id)

#%%time
def retrieve_data(s3_url):
    with fs.open(s3_url, 'rb') as compressed_data: # using s3fs
        buffer = ncd.blosc.decompress(compressed_data.read())

        dtype = "<f2"
        if "surface/PRES" in s3_url: 
            dtype = "<f4"

        chunk = np.frombuffer(buffer, dtype=dtype)
        
        entry_size = 150*150
        num_entries = len(chunk)//entry_size

        if num_entries == 1: # analysis file is 2d
            data_array = np.reshape(chunk, (150, 150))
        else:
            data_array = np.reshape(chunk, (num_entries, 150, 150))

    return data_array


data = retrieve_data(data_url + fcst_chunk_id)
print(data.shape)

gridpoint_forecast = data[:, nearest_point.in_chunk_y, nearest_point.in_chunk_x]

plt.title('HRRR Zarr Example %s Forecast' %(var))
plt.xlabel('Forecast Hour')
plt.ylabel(var)
plt.plot(gridpoint_forecast)


# Slice to get only the first 24 hours

In [ ]:
one_day_forecast = data[:24, nearest_point.in_chunk_y, nearest_point.in_chunk_x]

plt.title(f'HRRR Zarr Example {var} Forecast for {date}')
plt.xlabel('Forecast Hour')
plt.ylabel(var)
plt.plot(one_day_forecast)
plt.show()

print(one_day_forecast)

# Visual: single date and single variable TMP

In [ ]:
H = Herbie(
    "2023-07-12",
    model="hrrr",
    product="sfc",
    fxx=14,
)

# Show available products
print(H.PRODUCTS)

ds = H.xarray("TMP:2 m above")

#
##
###
####
#####
#plot
fig = plt.figure(figsize=[10, 8])
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

ax.add_feature(cfeature.STATES)

p = ax.pcolormesh(
    ds.longitude,
    ds.latitude,
    ds.t2m, 
    transform=ccrs.PlateCarree(),
    vmin=266, 
    vmax=310,
)
cbar = plt.colorbar(
    p,
    ax=ax,
    orientation="horizontal",
    pad=0.01,
    shrink=0.8,
)

#ax.set_extent([-125, -102, 32, 42])

ax.set_title(f"Variable: Temperature - Date: 2023-07-12")

plt.show()
######
#####
####
###
##
#
##
###
####
#####
######

# Plot Function 

In [ ]:
def plot_variable(var_name, date):
    H = Herbie(
        date,
        model="hrrr",
        product="sfc",
        fxx=14,
    )
    
    # Assuming H is your Herbie object
    ds = H.xarray(var_name)

    # Extract the first (and presumably only) data variable name
    data_var = list(ds.data_vars)[0]

    # Fetch the actual data for the variable
    data = ds[data_var]

    fig = plt.figure(figsize=[10, 8])
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
    ax.add_feature(cfeature.STATES)

    p = ax.pcolormesh(
        ds.longitude,
        ds.latitude,
        data, 
        transform=ccrs.PlateCarree(),
        vmin=data.min(), 
        vmax=data.max(),
    )
    
    cbar = plt.colorbar(
        p,
        ax=ax,
        orientation="horizontal",
        pad=0.01,
        shrink=0.8,
    )
    
    ax.set_extent([-125, -102, 32, 42])
    
    #ax.set_title(f"Variable: {var_name} - Date: {date}")
    fig.text(0.5, 0.12, f"Variable: {var_name} - Date: {date}", ha='center')
    
    plt.show()

# Example usage: ONLY CHANGE THIS
# https://www.nco.ncep.noaa.gov/pmb/products/hrrr/hrrr.t00z.wrfprsf00.grib2.shtml
plot_variable("RH:2 m above", "2023-07-12")


# Prepare data - daily fire progression

In [ ]:
import datetime
from datetime import timedelta
import time

# Open the shapefile and create a GeoDataFrame
gdf = gpd.read_file("C:\\PATH\\TO\\FILE\\Dixie_DailyFireProg_VIIRS.shp")

# Convert the timestamp column to datetime type
gdf['t'] = pd.to_datetime(gdf['t'])

# Group the GeoDataFrame by 't' and 'FireName' and apply unary_union to combine the polygons
grouped = gdf.groupby(['t', 'fireid'])

# Create a new GeoDataFrame with the combined polygons and additional attributes
gdf_combined = grouped.agg({'geometry': unary_union, 'farea': 'first'}).reset_index()

# Create a GeoDataFrame from the combined polygons
gdf_combined = gpd.GeoDataFrame(gdf_combined, geometry='geometry')

# Sort the GeoDataFrame by the timestamp
gdf_sorted = gdf_combined.sort_values('t')

# Select rows with a timestamp of 12:00:00
gdf_1200 = gdf_sorted[gdf_sorted['t'].dt.time == datetime.time(0, 0, 0)]

# Calculate the change in "farea" for each FireName
gdf_1200['farea_change'] = gdf_1200.groupby('fireid')['farea'].diff()

gdf_1200['farea_change'] = gdf_1200['farea_change'].clip(lower=0)

# Set values of 'farea_change' to zero if they are less than 50 acre change
#gdf_1200.loc[gdf_1200['farea_change'] < 0.005, 'farea_change'] = 0

# Replace NaN values in "farea_change" with 0
gdf_1200['farea_change'] = gdf_1200['farea_change'].fillna(0)

gdf_1200_pt = gdf_1200.copy()

gdf_1200_pt['geometry'] = gdf_1200_pt['geometry'].centroid

gdf_1200_pt = gpd.GeoDataFrame(gdf_1200_pt, geometry='geometry')

gdf_1200_pt['Latitude'] = gdf_1200_pt['geometry'].y

gdf_1200_pt['Longitude'] = gdf_1200_pt['geometry'].x

gdf_1200_pt['farea_change_shifted'] = gdf_1200_pt.groupby('fireid')['farea_change'].shift(-1)

gdf_1200_pt['farea_change_shifted'] = gdf_1200_pt['farea_change_shifted'].fillna(0)

columns_to_join = ['fireid', 't', 'n_pixels','n_newpixel','meanfrp','fperim','duration']

gdf_1200_pt = pd.merge(gdf_1200_pt, gdf[columns_to_join], on=['fireid', 't'], how='left')

# Convert the timestamp column to datetime type
gdf_1200_pt['t'] = pd.to_datetime(gdf_1200_pt['t'])

FireNameList = gdf_1200_pt["fireid"].unique()
print(FireNameList)

gdf_1200_pt

# Download - hrrrzarr

In [ ]:
# https://home.chpc.utah.edu/~u0553130/Brian_Blaylock/HRRR_archive/hrrr_prs_table_f00-f01.html

# #FireNameList = gdf_1200_pt["FireName"].unique()
# FireNameList = ["AUGUST COMPLEX","RED SALMON COMPLEX","OAK","ABNEY","CAMERON PEAK","EAST TROUBLESOME","MULLEN",
#                 "MCLEOD", "CRESCENT MOUNTAIN", "CARR","FERGUSON","WATSON CREEK",
#                "CHETCO BAR","BALD MOUNTAIN","DOLLAR RIDGE","COUGAR CREEK","TAYLOR CREEK","MILES",
#                "CAMP","WALKER","DECKER","CREEK","CASTLE","EAST FORK","BUSH","BIGHORN",
#                "CEDAR CREEK", "COW CANYON","BEAR"]

# ~~~change 'output_file' below to export csv for each fire~~~ 

# Hour: [00-23]z must be in the two digit format  i.e. 06 or 19
# noon
hr = '12'

for i in range(len(FireNameList)):
    
    try:
        selected_fire = FireNameList[i]
        gdf_singleFire = gdf_1200_pt[gdf_1200_pt["fireid"] == selected_fire]

        LatitudeLIST = gdf_singleFire['Latitude'].tolist()
        LongitudeLIST = gdf_singleFire["Longitude"].tolist()
        DateLIST = gdf_singleFire["t"].tolist()
        print(f"The number of days: {len(LatitudeLIST)}")

        #
        ##
        ###
        ####
        for i in range(len(gdf_singleFire)):

            # point_lat = 40.7608
            # point_lon = -113.8910

            # # Date: [YYYYMMDD] must be in this format
            # date = '20230718'

            #location: Example is Salt Lake City (SLC). Can only do coordinates in area that is covered by the HRRR CONUS grid.
            point_lat = LatitudeLIST[i]
            point_lon = LongitudeLIST[i]
            date_string = DateLIST[i]

            from datetime import datetime
            from dateutil.relativedelta import relativedelta

            date_object = datetime.strptime(date_string.strftime('%Y-%m-%d'), "%Y-%m-%d")
            date = date_object.strftime("%Y%m%d")#+ relativedelta(years=4)

            # Date: [YYYYMMDD] must be in this format. Example is for Jan 8, 2021
            #date = '20180713'
            print(type(date))
            print(date)

            # Temperature (K)
            level = 'surface'
            var = 'TMP'
            data_TMP = f'hrrrzarr/sfc/{date}/{date}_{hr}z_fcst.zarr/{level}/{var}/{level}/{var}/'
            
            # Wind Speed (gust) (m/s)
            level = 'surface'
            var = 'GUST'
            data_GUST = f'hrrrzarr/sfc/{date}/{date}_{hr}z_fcst.zarr/{level}/{var}/{level}/{var}/'

            # Surface Roughness (m)
            level = 'surface'
            var = 'SFCR'
            data_SFCR = f'hrrrzarr/sfc/{date}/{date}_{hr}z_fcst.zarr/{level}/{var}/{level}/{var}/'

            # Ground Heat Flux (W/m^2)
            level = 'surface'
            var = 'GFLUX'
            data_GFLUX = f'hrrrzarr/sfc/{date}/{date}_{hr}z_fcst.zarr/{level}/{var}/{level}/{var}/'

            # Plant Canopy Surface Water (kg/m^2)
            level = 'surface'
            var = 'CNWAT'
            data_CNWAT = f'hrrrzarr/sfc/{date}/{date}_{hr}z_fcst.zarr/{level}/{var}/{level}/{var}/'

            # Frictional Velocity (m/s)
            level = 'surface'
            var = 'FRICV'
            data_FRICV = f'hrrrzarr/sfc/{date}/{date}_{hr}z_fcst.zarr/{level}/{var}/{level}/{var}/'
    
            # Relative Humidity (%)
            level = '2m_above_ground'
            var = 'RH'
            data_RH = f'hrrrzarr/sfc/{date}/{date}_{hr}z_fcst.zarr/{level}/{var}/{level}/{var}/'

            # Dew Point Temperature (K)
            level = '2m_above_ground'
            var = 'DPT'
            data_DPT = f'hrrrzarr/sfc/{date}/{date}_{hr}z_fcst.zarr/{level}/{var}/{level}/{var}/'

            # Specific Humidity (kg/kg)
            level = '2m_above_ground'
            var = 'SPFH'
            data_SPFH = f'hrrrzarr/sfc/{date}/{date}_{hr}z_fcst.zarr/{level}/{var}/{level}/{var}/'
          
            # Potential Temperature (K)
            level = '2m_above_ground'
            var = 'POT'
            data_POT = f'hrrrzarr/sfc/{date}/{date}_{hr}z_fcst.zarr/{level}/{var}/{level}/{var}/'
            
            # U-Component of Wind (m/s)
            level = '10m_above_ground'
            var = 'UGRD'
            data_UGRD = f'hrrrzarr/sfc/{date}/{date}_{hr}z_fcst.zarr/{level}/{var}/{level}/{var}/'
       
            # V-Component of Wind (m/s)
            level = '10m_above_ground'
            var = 'VGRD'
            data_VGRD = f'hrrrzarr/sfc/{date}/{date}_{hr}z_fcst.zarr/{level}/{var}/{level}/{var}/'
            
            # Vegetation Type (Integer(0-13))
            level = 'surface'
            var = 'VGTYP'
            data_VGTYP = f'hrrrzarr/sfc/{date}/{date}_{hr}z_fcst.zarr/{level}/{var}/{level}/{var}/'

            fs = s3fs.S3FileSystem(anon=True)

            chunk_index = xr.open_zarr(s3fs.S3Map("s3://hrrrzarr/grid/HRRR_chunk_index.zarr", s3=fs))
            ###
            ###
            ###
            import cartopy.crs as ccrs

            # This is the projection the HRRR grid uses.
            projection = ccrs.LambertConformal(central_longitude=262.5, 
                                               central_latitude=38.5, 
                                               standard_parallels=(38.5, 38.5),
                                                globe=ccrs.Globe(semimajor_axis=6371229,
                                                                 semiminor_axis=6371229))

            x, y = projection.transform_point(point_lon, point_lat, ccrs.PlateCarree())

            nearest_point = chunk_index.sel(x=x, y=y, method="nearest")
            fcst_chunk_id = f"0.{nearest_point.chunk_id.values}"
            #print(fcst_chunk_id)
            ###
            ###
            ###

            #%%time
            def retrieve_data(s3_url):
                with fs.open(s3_url, 'rb') as compressed_data: # using s3fs
                    buffer = ncd.blosc.decompress(compressed_data.read())

                    dtype = "<f2"
                    if "surface/PRES" in s3_url: # surface/PRES is the only variable with a larger data type
                        dtype = "<f4"

                    chunk = np.frombuffer(buffer, dtype=dtype)

                    entry_size = 150*150
                    num_entries = len(chunk)//entry_size

                    if num_entries == 1: # analysis file is 2d
                        data_array = np.reshape(chunk, (150, 150))
                    else:
                        data_array = np.reshape(chunk, (num_entries, 150, 150))

                return data_array
            
            try:
                data = retrieve_data(data_TMP + fcst_chunk_id)
                gridpoint_forecast = data[:, nearest_point.in_chunk_y, nearest_point.in_chunk_x]
                gdf_singleFire.at[gdf_singleFire.index[i], 'TMP'] = gridpoint_forecast[0]
            except:
                gdf_singleFire.at[gdf_singleFire.index[i], 'TMP'] = "NOTAVAILABLE"

            try:
                data = retrieve_data(data_GUST + fcst_chunk_id)
                gridpoint_forecast = data[:, nearest_point.in_chunk_y, nearest_point.in_chunk_x]
                gdf_singleFire.at[gdf_singleFire.index[i], 'GUST'] = gridpoint_forecast[0]
            except:
                gdf_singleFire.at[gdf_singleFire.index[i], 'GUST'] = "NOTAVAILABLE"
            
            try:
                data = retrieve_data(data_SFCR + fcst_chunk_id)
                gridpoint_forecast = data[:, nearest_point.in_chunk_y, nearest_point.in_chunk_x]
                gdf_singleFire.at[gdf_singleFire.index[i], 'SFCR'] = gridpoint_forecast[0]
            except:
                gdf_singleFire.at[gdf_singleFire.index[i], 'SFCR'] = "NOTAVAILABLE"
                
            try:
                data = retrieve_data(data_GFLUX + fcst_chunk_id)
                gridpoint_forecast = data[:, nearest_point.in_chunk_y, nearest_point.in_chunk_x]
                gdf_singleFire.at[gdf_singleFire.index[i], 'GFLUX'] = gridpoint_forecast[0]
            except:
                gdf_singleFire.at[gdf_singleFire.index[i], 'GFLUX'] = "NOTAVAILABLE"

            try:                
                data = retrieve_data(data_CNWAT + fcst_chunk_id)
                gridpoint_forecast = data[:, nearest_point.in_chunk_y, nearest_point.in_chunk_x]
                gdf_singleFire.at[gdf_singleFire.index[i], 'CNWAT'] = gridpoint_forecast[0]
            except:
                gdf_singleFire.at[gdf_singleFire.index[i], 'CNWAT'] = "NOTAVAILABLE"
                
            try:
                data = retrieve_data(data_FRICV + fcst_chunk_id)
                gridpoint_forecast = data[:, nearest_point.in_chunk_y, nearest_point.in_chunk_x]
                gdf_singleFire.at[gdf_singleFire.index[i], 'FRICV'] = gridpoint_forecast[0]
            except:
                gdf_singleFire.at[gdf_singleFire.index[i], 'FRICV'] = "NOTAVAILABLE"

            try:
                data = retrieve_data(data_RH + fcst_chunk_id)
                gridpoint_forecast = data[:, nearest_point.in_chunk_y, nearest_point.in_chunk_x]
                gdf_singleFire.at[gdf_singleFire.index[i], 'RH'] = gridpoint_forecast[0]
            except:
                gdf_singleFire.at[gdf_singleFire.index[i], 'RH'] = "NOTAVAILABLE"

            try:
                data = retrieve_data(data_DPT + fcst_chunk_id)
                gridpoint_forecast = data[:, nearest_point.in_chunk_y, nearest_point.in_chunk_x]
                gdf_singleFire.at[gdf_singleFire.index[i], 'DPT'] = gridpoint_forecast[0]
            except:
                gdf_singleFire.at[gdf_singleFire.index[i], 'DPT'] = "NOTAVAILABLE"
                
            try:     
                data = retrieve_data(data_SPFH + fcst_chunk_id)
                gridpoint_forecast = data[:, nearest_point.in_chunk_y, nearest_point.in_chunk_x]
                gdf_singleFire.at[gdf_singleFire.index[i], 'SPFH'] = gridpoint_forecast[0]
            except:
                gdf_singleFire.at[gdf_singleFire.index[i], 'SPFH'] = "NOTAVAILABLE"
               
            try:   
                data = retrieve_data(data_POT + fcst_chunk_id)
                gridpoint_forecast = data[:, nearest_point.in_chunk_y, nearest_point.in_chunk_x]
                gdf_singleFire.at[gdf_singleFire.index[i], 'POT'] = gridpoint_forecast[0]
            except:
                gdf_singleFire.at[gdf_singleFire.index[i], 'POT'] = "NOTAVAILABLE"                
                
            try:
                data = retrieve_data(data_UGRD + fcst_chunk_id)
                gridpoint_forecast = data[:, nearest_point.in_chunk_y, nearest_point.in_chunk_x]
                gdf_singleFire.at[gdf_singleFire.index[i], 'UGRD'] = gridpoint_forecast[0]
            except:
                gdf_singleFire.at[gdf_singleFire.index[i], 'UGRD'] = "NOTAVAILABLE"                

            try:    
                data = retrieve_data(data_VGRD + fcst_chunk_id)
                gridpoint_forecast = data[:, nearest_point.in_chunk_y, nearest_point.in_chunk_x]
                gdf_singleFire.at[gdf_singleFire.index[i], 'VGRD'] = gridpoint_forecast[0]
            except:
                gdf_singleFire.at[gdf_singleFire.index[i], 'VGRD'] = "NOTAVAILABLE"                

            try:    
                data = retrieve_data(data_VGTYP + fcst_chunk_id)
                gridpoint_forecast = data[:, nearest_point.in_chunk_y, nearest_point.in_chunk_x]
                gdf_singleFire.at[gdf_singleFire.index[i], 'VGTYP'] = gridpoint_forecast[0]
            except:
                gdf_singleFire.at[gdf_singleFire.index[i], 'VGTYP'] = "NOTAVAILABLE"                
                  
            print(i)

        # Define the output file path and name
        output_file = f"C:\\PATH\\TO\\OUTPUT\\FILE\\HRRR_{selected_fire}.csv"
    
        # Save the GeoDataFrame as a CSV file
        gdf_singleFire.to_csv(output_file, index=False)
        ####
        ###
        ##
        #
    except Exception as e:
        
        print(f"An error occurred for {selected_fire}:")
        traceback.print_exc()
        
        continue

print("Done!")

# GIF of gust 

In [ ]:
import matplotlib.animation as animation
from datetime import timedelta, datetime

# Define start and end dates
start_date = datetime(2023, 8, 1)
end_date = datetime(2023, 8, 7)

# Define a function to update the plot for a given day
def update(day):
    ax.clear()
    ax.add_feature(cfeature.STATES)
    date = start_date + timedelta(days=day)
    H = Herbie(
        date.strftime("%Y-%m-%d"),
        model="hrrr",
        product="sfc",
        fxx=0,
    )
    #ds = H.xarray("TMP:2 m above")
    ds = H.xarray("GUST:surface")
    p = ax.pcolormesh(
        ds.longitude,
        ds.latitude,
        ds.gust, 
        transform=ccrs.PlateCarree(),
        vmin=0, 
        vmax=20,
    )
    ax.set_extent([-125, -102, 32, 42])
    return p,

# Create a new figure
fig = plt.figure(figsize=[8, 5])
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

# Create an animation
ani = animation.FuncAnimation(fig, update, frames=(end_date-start_date).days, interval=200)

# Save the animation as a GIF
ani.save('C:\\PATH\\TO\\OUTPUT\\WEATHER_GIF.gif', writer='imagemagick')

# Close the figure
plt.close(fig)